### Establecer la localización más óptima para las oficinas de una empresa de video juegos con los siguientes condiciones:

##### -Es una compañía de software que crea videojuegos.
##### -Tiene 50 empleados. Los roles dentro de la nueva compañía son: 20 desarrolladores, 20 diseñadores/creativos/UX-UI y 10 ejecutivos/managers.
##### -Debe haber ingenieros de software trabajando alrededor, así como diseñadores, desarrolladores y ejecutivos para cubrir los intereses del equipo.
##### -Debe haber un buen ratio de empresas grandes vs. startups.
##### -Se prefieren empresas de reciente creación.




In [24]:
# Importar librerias.
import pandas as pd
import numpy as np
import pymongo
import geopandas

In [2]:
# Cargar la base de de MongoDB.
MongoClient = pymongo.MongoClient # Llamar a Mongo (conectar con el cliente).
client = MongoClient() # Guardar Mongo en variable cliente.
db = client.companies # Dentro de mongo conecto con base de dato company.

In [3]:
# Imprimo todas las categorias para ver que tengo.
cursor = db.companies.find({'category_code':{'$exists': 'true'}}) # Creo mi cursor para iterar secuencialmente los resultados obtenidos de la query.
categories = list(map(lambda x:x['category_code'], list(cursor))) # Creo función lambda para que me consiga todos mis category_code que debo convertir en lista ya que es un objeto.

category_codes = [] # Asigno y creo lista vacia a category_codes.
for element in categories: # Para cada elemento en categories 
    if element not in category_codes: # si el elemento no esta en category_codes
        category_codes.append(element) # Entonces mi lista vacia de category_codes le voy a .append(agregar) el elemento
    else:                              # si no.
        pass                           # pasa.
print(category_codes)   # Display todas las catergory_codes de mi lista.  

['enterprise', 'software', 'web', 'news', 'network_hosting', 'games_video', 'music', 'mobile', 'social', 'search', 'messaging', 'advertising', 'security', 'photo_video', 'finance', 'ecommerce', 'travel', 'hardware', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


In [4]:
# Creando cursor con los parametros que necesito
# Pongo deadpooled_year: None ya que no me interesa que aparezca en mi data frame las empresas que estan o irán a quiebra.
#companies_interest_cursor = list(db.companies.find({"deadpooled_year": None},
                                       #{"_id":0,"name":1,"category_code":1,"number_of_employees":1,
                                       #"total_money_raised":1,"founded_year":1,
                                       #"offices.latitude":1,"offices.longitude":1}))


In [8]:
companies_interest_cursor = db.companies.find({"$and": 
                               [{"offices.latitude": {"$exists": True}}, {"offices.latitude": {"$ne": None}},
                                {"offices.longitude": {"$exists": True}}, {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}}, {"category_code": {"$ne": None}},
                                {"name": {"$exists": True}}, {"name": {"$ne": None}},
                                {"deadpooled_year": {"$ne": None}},
                                {"founded_year": {"$gte": 2008}}, 
                                {"category_code": {"$ne": ["enterprise", "news", "music", "social", "mobile",
                                                          "advertising", "security", "messaging", "finance",
                                                          "ecommerce", "travel", "public_relations","other", 
                                                           "real_estate","semiconductor", "analytics", "health",
                                                          "legal", "sports", "education", "consulting",
                                                           "transportation", "None", "hospitality", "fashion"
                                                          "nonprofit", "automotive", "design", "manufacturing"
                                                          "government", "local", "medical"]}}]}, 
                              
                                {"_id":1, 
                                 "category_code":1,
                                 "founded_year":1,
                                 "ipo.valuation_amount":1, 
                                 "name":1,
                                 "number_of_employees":1,
                                 "offices.latitude":1,
                                 "offices.longitude":1,
                                 "total_money_raised":1})

In [9]:
df_companies = pd.DataFrame(companies_interest_cursor) # Creando dataframe con companies_interest_cursor

display(df_companies.shape) # Mostrando el tamaño de mi dataframe.

(118, 7)

In [10]:
df_companies.head(10) # Mostrar las 10 primeras filas de mi dataframe.

,_id,category_code,founded_year,name,number_of_employees,offices,total_money_raised
0,52cdef7c4bab8bd6752980bb,enterprise,2010,Needium,15.0,"[{'latitude': 45.5049305, 'longitude': -73.568...",$1M
1,52cdef7c4bab8bd6752982d4,advertising,2013,Advaliant,60.0,"[{'latitude': 43.646599, 'longitude': -79.4030...",$100k
2,52cdef7c4bab8bd6752982ec,web,2008,8aweek,NaN,"[{'latitude': 37.09024, 'longitude': -95.712891}]",$0
3,52cdef7c4bab8bd675298367,web,2008,JackPot Rewards,NaN,"[{'latitude': 42.337037, 'longitude': -71.2517...",$16.7M
4,52cdef7c4bab8bd6752984a6,web,2008,CityIN,11.0,"[{'latitude': 35.86166, 'longitude': 104.195397}]",$0
5,52cdef7c4bab8bd675298577,mobile,2008,Tapioca Mobile,NaN,"[{'latitude': 32.88945, 'longitude': -117.1669...",$5M
6,52cdef7c4bab8bd6752985ac,mobile,2011,Moblica,NaN,"[{'latitude': 32.0554, 'longitude': 34.7595}]",$6M
7,52cdef7c4bab8bd675298693,web,2008,Wakozi,5.0,"[{'latitude': 40.755409, 'longitude': -73.9927...",$0
8,52cdef7c4bab8bd67529881b,web,2008,Twitlinks,NaN,"[{'latitude': -33.329799, 'longitude': 151.505...",$0
9,52cdef7d4bab8bd675298893,web,2008,StrawPoll,3.0,"[{'latitude': 36.0073, 'longitude': -78.9206}]",$0


In [11]:
# cambio el nombre de la columna category_code a category y offices a coordenadas(cords) para visualizarlo mejor.
df_companies = df_companies.rename(index=str, columns={"category_code": "category", "offices": "coords"})
# cambio el orden de las columnas para visualizar mejor el dataframe.
df_companies = df_companies.reindex(['category', 'name', 'coords', 'number_of_employees', 'founded_year', 'total_money_raised'],axis=1)
df_companies.head() # Mostrar las 5 primeras filas de mi dataframe.


,category,name,coords,number_of_employees,founded_year,total_money_raised
0,enterprise,Needium,"[{'latitude': 45.5049305, 'longitude': -73.568...",15.0,2010,$1M
1,advertising,Advaliant,"[{'latitude': 43.646599, 'longitude': -79.4030...",60.0,2013,$100k
2,web,8aweek,"[{'latitude': 37.09024, 'longitude': -95.712891}]",NaN,2008,$0
3,web,JackPot Rewards,"[{'latitude': 42.337037, 'longitude': -71.2517...",NaN,2008,$16.7M
4,web,CityIN,"[{'latitude': 35.86166, 'longitude': 104.195397}]",11.0,2008,$0


In [12]:
df_companies.dtypes # Veo de que tipo son mis columnas.

category                object
name                    object
coords                  object
number_of_employees    float64
founded_year             int64
total_money_raised      object
dtype: object

In [13]:
#Creo funcion para transformar datos.

def float_to_int(flt):    #Transforma un dato tipo float a uno tipo int.
    integer = int(flt)
    return integer

In [14]:
df_companies['number_of_employees'].fillna(0, inplace=True) # Relleno todos los NAN en columna number_of_employees.
df_companies['founded_year'].fillna(0, inplace=True) # Relleno todos los NAN en columna founded_year.


In [18]:
# Aplico la función para transformar mis datos a enteros.
df_companies['number_of_employees'] = df_companies['number_of_employees'].apply(float_to_int)
df_companies['founded_year'] = df_companies['founded_year'].apply(float_to_int)

In [19]:
df_companies.dtypes # Veo de que tipo son mis columnas para ver si el cambio se aplico.

category               object
name                   object
coords                 object
number_of_employees     int64
founded_year            int64
total_money_raised     object
dtype: object

In [20]:
# Crear unos bins para ver el tamaño de la empresa segun los'number_of_employees': startup, small, medium y big

bins = [0,10,100,1000,100000] # Creo categorias de el bin
labels = ["startup","small","medium","big"] # Creo nombre para las categorias del bin
df_companies['tamaño_empresa'] = pd.cut(df_companies['number_of_employees'], bins=bins, labels=labels)
df_companies.dropna(subset=['tamaño_empresa'], inplace=True)
df_companies.groupby(df_companies.tamaño_empresa)['name'].nunique() 


tamaño_empresa
startup    74
small       9
medium      0
big         0
Name: name, dtype: int64

In [21]:
df_companies.head()

,category,name,coords,number_of_employees,founded_year,total_money_raised,tamaño_empresa
0,enterprise,Needium,"[{'latitude': 45.5049305, 'longitude': -73.568...",15,2010,$1M,small
1,advertising,Advaliant,"[{'latitude': 43.646599, 'longitude': -79.4030...",60,2013,$100k,small
4,web,CityIN,"[{'latitude': 35.86166, 'longitude': 104.195397}]",11,2008,$0,small
7,web,Wakozi,"[{'latitude': 40.755409, 'longitude': -73.9927...",5,2008,$0,startup
9,web,StrawPoll,"[{'latitude': 36.0073, 'longitude': -78.9206}]",3,2008,$0,startup


In [23]:
df_companies.to_json('prueba.json', orient="records", lines=True)